## Variant 3 (Fear Greed + Markov Switching)

In [7]:
import numpy as np
import pandas as pd
import scipy as sp
import yfinance as yf
import pypfopt 
import matplotlib.pyplot as plt

from pypfopt import black_litterman, plotting, objective_functions, risk_models
from pypfopt import BlackLittermanModel
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.expected_returns import mean_historical_return
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression

In [9]:
tickers = ["TSLA", "NVDA", "AMZN", "META", "MSFT", "AAPL", "AMD"]
start_date = '2013-01-01'
end_date = '2024-01-01'

data = yf.download(tickers, start=start_date, end=end_date)
data.to_csv("./Data/tickers.csv")

closing_prices = data["Adj Close"]
returns = np.log(closing_prices / closing_prices.shift(1).dropna())

def get_market_prices_and_caps(tickers, start_date, end_date):
    market_prices = yf.download("VOO", start=start_date, end=end_date)["Adj Close"]

    market_caps = {}
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        market_caps[ticker] = stock.info.get("marketCap", None)

    return market_prices, market_caps

closing_prices.tail(10)

[*********************100%***********************]  7 of 7 completed


Ticker,AAPL,AMD,AMZN,META,MSFT,NVDA,TSLA
Date,,,,,,,
2023-12-15 00:00:00+00:00,196.606827,139.149994,149.970001,333.645508,367.978333,48.876141,253.500000
2023-12-18 00:00:00+00:00,194.934998,138.899994,154.070007,343.308563,369.884033,50.062805,252.080002
2023-12-19 00:00:00+00:00,195.979889,140.149994,153.789993,349.026764,370.489532,49.589939,257.220001
2023-12-20 00:00:00+00:00,193.880188,135.470001,152.119995,347.950836,367.869110,48.097359,247.139999
2023-12-21 00:00:00+00:00,193.730881,139.910004,153.839996,352.742523,370.767487,48.976116,254.500000
2023-12-22 00:00:00+00:00,192.656189,139.600006,153.419998,352.045227,371.799713,48.816162,252.539993
2023-12-26 00:00:00+00:00,192.108856,143.410004,153.410004,353.479706,371.879120,49.265030,256.609985
2023-12-27 00:00:00+00:00,192.208359,146.070007,153.339996,356.468323,371.293518,49.402992,261.440002
2023-12-28 00:00:00+00:00,192.636261,148.759995,153.380005,356.956451,372.494507,49.507965,253.179993


In [11]:
# def plot_log_returns(returns):
#     plt.figure(figsize=(7,4))
#     plt.plot(returns.index, returns, label='Log Returns')
#     plt.title('Vanguard S&P 500 (VOO) Log Returns')
#     plt.xlabel('Date')
#     plt.ylabel('Log Returns')
#     plt.legend(loc='upper left')
#     plt.grid()
#     plt.show()

# plot_log_returns(returns)

In [8]:
def fit_markov_switching(data):
    model = MarkovRegression(data, k_regimes=2, switching_variance=True)
    results = model.fit

    return results

results = fit_markov_switching(data)

def set_view_confidences_for_views(results, markov_relative_views, threshold_bull=0.6, threshold_bear=0.4):
    smoothed_probs = results.smoothed_marginal_probabilities
    
    view_confidences = []
    
    # check views matches a valid range
    if len(markov_relative_views) > len(smoothed_probs[0]):
        raise ValueError("Number of views exceeds the number of time steps in smoothed probabilities.")
    
    # Iterate over views and assign confidences
    for idx in range(len(markov_relative_views)):
        bull_prob = smoothed_probs[0][idx]
        bear_prob = smoothed_probs[1][idx]
        
        if bull_prob > threshold_bull:
            confidence = bull_prob
        elif bear_prob > threshold_bear:
            confidence = bear_prob
        else:
            confidence = max(bull_prob, bear_prob)
        
        view_confidences.append(confidence)
    
    return view_confidences

markov_adjusted_confidences = set_view_confidences_for_views(results, markov_relative_views, threshold_bull=0.6, threshold_bear=0.4)
markov_adjusted_confidences

c:\Users\Chantal.Maskell\Documents\Projects\Black_Litterman_Portfolio_Optimisation\.venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


ValueError: Must have univariate endogenous data.